In [ ]:
import numpy as np

#从用户输入中接收参评数目和指标数目，并将输入的字符串转换为数值
print("请输入参评数目：")
n=int(input())
print("请输入指标数目：")
m=int(input())

#接收用户输入的类型矩阵，该矩阵指示了每个指标的类型
print("请输入类型矩阵：1.极大型，2.极小型，3.中间型，4.区间型")
kind=input().split(" ")#将输入的字符串按空格割裂，形成列表

#接收用户输入的矩阵并转换为numpy数组
print("请输入矩阵：")
A=np.zeros(shape=(n,m))
for i in range(n):
    A[i]=input().split(" ")#接收每行输入的数据
    A[i]=list(map(float,A[i]))#将每行数据转换为浮点数
print("输入矩阵为：\n{}".format(A))

#极小型指标转化为极大型指标的函数
def minTomax(maxx,x):
    x=list(x)#将输入的数据转换为列表
    ans=[[(maxx-e)] for e in x]#计算最大值与每个指标的差，并将其放入新列表中
    return np.array(ans)#将列表转换为numpy数组并返回

def midTomax(bestx,x):
    x=list(x)#将输入的数据转换为列表
    h=[abs(e-bestx) for e in x]#计算每个指标与最优值的差
    M=max(h)#找出最大差
    if M==0:
        M=1#如果最大差为0，则将其设为1
    ans=[[(1-e)/M] for e in h]#计算每个指标与最大差的比例
    return np.array(ans)#将列表转换为numpy数组并返回

def regTomax(lowx,highx,x):
    x=list(x)#将输入的数据转换为列表
    M=max(lowx-min(x),max(x)-highx)#计算指标值超出区间的最大距离
    if M==0:
        M=1#防止最大距离维0的情况
    ans=[]
    for i in range(len(x)):
        if x[i]<=lowx:
            ans.append([(1-(lowx-x[i]))/M])#如果指标值小于下限，则计算其与下限的比例
        elif x[i]>=highx:
            ans.append([(1-(x[i]-highx))/M])#如果指标值大于上限，则计算其与上限的比例
        else:
            ans.append([1])#如果指标值在区间内，则其比例为1
    return np.array(ans)#将列表转换为numpy数组并返回

#统一指标类型，将所有指标转化为极大型指标
X=np.zeros(shape=(n,1))
for i in range(m):
    if kind[i]=="1":#如果当前指标为极大型，则直接使用原值
        v=np.array([A[:,i]])
    elif kind[i]=="2":#如果当前指标为极小型，则使用极小型转化的函数
        v=minTomax(max(A[:,i]),A[:,i])
    elif kind[i]=="3":#如果当前指标为中间型，则使用中间型转化的函数
        print("类型三，请输入最优值：")
        bestA=eval(input())
        v=midTomax(bestA,A[:,i])
    elif kind[i]=="4":#如果当前指标为区间型，则使用区间型转化的函数
        print("类型四，请输入下限：")
        lowA=eval(input())
        print("类型四，请输入上限：")
        highA=eval(input())
        v=regTomax(lowA,highA,A[:,i])
    if i==0:
        X=v.reshape(-1,1)#如果是第一个指标，直接替换X数组
    else:
        X=np.hstack((X,v.reshape(-1,1)))#如果是其他指标，则将转化后的指标添加到X数组中
print("统一指标后的矩阵为：\n{}".format(X))
#对统一指标后的矩阵X进行标准化处理
X=X.astype("float")#确保X矩阵的数据类型为浮点数
for j in range(m):
    X[:,j]=X[:,j]/np.sqrt(np.sum(np.square(X[:,j])))#对每个指标进行标准化处理
print("标准化后的矩阵为：\n{}".format(X))

#最大值最小值距离的计算
x_max=np.max(X,axis=0)#计算每列的最大值
x_min=np.min(X,axis=0)#计算每列的最小值
d_z=np.sqrt(np.sum(np.square((X-np.tile(x_max,(n,1)))),axis=1))#计算每个参评对象与最优情况的距离d+
d_f=np.sqrt(np.sum(np.square((X-np.tile(x_min,(n,1)))),axis=1))#计算每个参评对象与最劣情况的距离d-
print("每个指标的最大值：",x_max)
print("每个指标的最小值：",x_min)
print("d+：",d_z)
print("d-：",d_f)
#计算每个参评对象的综合得分
s=d_f/(d_z+d_f)#根据d+和d-计算综合得分
Score=100*s/sum(s)#将得分s转换为百分制，便于比较
for i in range(len(Score)):
    print("第{}个参评对象的得分为：{}".format(i+1,Score[i]))